In [1]:
# !nvidia-smi

import os
import sys
import torch
import argparse
from utils.utils import predict, show
from model import model as module_arch
from model import model as module_arch
from utils.parse_config import ConfigParser
from utils import dataloader as module_data, metric, utils

class TestModel:
    def __init__(self, resume_path):
        self.args = self._parse_arguments(resume_path)
        self.config = ConfigParser.from_args(self.args)
        self.logger = self.config.get_logger('test')
        self.data_loader = self._init_data_loader()
        self.model = self._init_model()

    @staticmethod
    def _parse_arguments(resume_path):
        parser = argparse.ArgumentParser(description='PyTorch Template')
        parser.add_argument('-c', '--config', default=None, type=str, help='config file path (default: None)')
        parser.add_argument('-r', '--resume', default=resume_path, type=str, help='path to latest checkpoint (default: None)')
        parser.add_argument('-d', '--device', default='cpu', type=str, help='indices of GPUs to enable (default: all)')

        # Check if in IPython environment
        if any("ipykernel" in arg for arg in sys.argv):
            sys.argv = sys.argv[:1]  # Keep only the script name

        return parser

    def _init_data_loader(self):
        self.config['dataloader']['args']['sample_data'] = True
        return self.config.init_obj('dataloader', module_data)

    def _init_model(self):
        model = self.config.init_obj('arch', module_arch, num_tag=self.data_loader.num_tag, path_lm=self.data_loader.path_lm)
        model.eval()
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        model = model.to(device)
        checkpoint = torch.load(self.config.resume, map_location='cpu')
        model.load_state_dict(checkpoint['state_dict'])
        return model.to(device)

    def test_prediction(self, text):
        tokens, out = utils.predict(self.model, text, self.data_loader.path_lm, self.data_loader.ids2spantag, self.data_loader.sent_length)
        tokens = [tk for tk in tokens if tk != self.data_loader.pad]
        print("|".join(tokens), "\n")
        [utils.show(x) for x in out]

/home/fame/anaconda3/envs/wangchanxner/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tester = TestModel('storage/best_model/model_best.pth')

Train : 71 sentences
Dev : 63 sentences
Test : 68 sentences
Max sents length: 512 tokens
num_vocab: 250002
num_tag: 105
num_span: 5
num_spantag: 417


Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaModel: ['lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
tester.test_prediction("เมื่อวันที่29ก.ค.55 สำนักข่าวบีบีซีรายงานว่า มหาวิทยาลัยออกซ์ฟอร์ดของอังกฤษได้แก้ไขกฎการแต่งกายภายในมหาวิทยาลัย")

<s>|เมื่อ|วันที่|29|ก|.|ค|.|55||สํานัก|ข่าว||บี|บี|ซี||รายงาน|ว่า|มหาวิทยาลัย||ออก|ซ์||ฟอร์ด|ของ||อังกฤษ|ได้||แก้ไข||กฎ|การ||แต่ง|กาย||ภายใน|มหาวิทยาลัย|</s> 

[2, 9]         date           วันที่29ก.ค.55
[12, 16]       media          บีบีซี
[26, 28]       country        อังกฤษ


: 